In [1]:
import os
os.environ["KERAS_BACKEND"] = "torch"

In [2]:
import keras
import keras.ops as K
from keras.layers import Input, Flatten, Dense
from keras.optimizers import Adam
from keras.metrics import BinaryAccuracy

# from keras.models import Sequential
from deel.lip.model import Sequential

from deel.lip.layers import (
    SpectralDense,
    SpectralConv2D,
    ScaledL2NormPooling2D,
    FrobeniusDense,
)
from deel.lip.activations import GroupSort, GroupSort2
from deel.lip.losses import HKR, KR, HingeMargin, MulticlassHKR, MulticlassKR

import numpy as np
import decomon

from data_processing import load_data, select_data_for_radius_evaluation_MNIST08
from radius_evaluation_tools import compute_binary_certificate, starting_point_dichotomy

In [3]:
from decomon.layers import DecomonLayer
from decomon.models import clone
from lipschitz_custom_tools import *
from decomon.perturbation_domain import BallDomain
from decomon import get_lower_noise, get_range_noise, get_upper_noise

In [4]:
import pdb

In [5]:
class DecomonGroupSort2(DecomonLayer):
    layer : GroupSort2
    increasing = True
    def get_affine_bounds(self, lower, upper):
        (W_low, b_low), (W_up, b_up) = affine_bound_groupsort_output_keras(lower, upper)
        W_low = K.transpose(W_low,(0,2,1))
        W_up = K.transpose(W_up,(0,2,1))
        return W_low, b_low, W_up, b_up

# Test GS2 decomon

## Cas 1 : z0 > 0

In [6]:
from deel.lip.model import Sequential

In [7]:
modelgs = Sequential([GroupSort2()])

In [8]:
input = K.expand_dims(K.array([2, -2], dtype='float32'), axis=0)
low = K.expand_dims(K.array([1, -3], dtype='float32'), axis=0)
up = K.expand_dims(K.array([3, -1], dtype='float32'), axis=0)
bounds =  K.expand_dims(K.concatenate((low,up), axis=0), axis=0)

In [9]:
modelgs(K.ones_like(input))

tensor([[1., 1.]], device='cuda:0')

In [10]:
bounds.shape

torch.Size([1, 2, 2])

In [11]:
decomon_model = clone(modelgs, mapping_keras2decomon_classes={GroupSort2:DecomonGroupSort2}, final_ibp=False, final_affine=True, method='forward-affine')  

2025-04-30 18:08:19.271947: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746029299.291706   17115 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746029299.297825   17115 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-30 18:08:19.318205: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
W_low, b_low, W_up, b_up = decomon_model.predict(bounds)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step


/home/aws_install/miniconda3/envs/k3torchenv/lib/python3.10/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['perturbation_domain_input_sequential']
Received: inputs=Tensor(shape=torch.Size([1, 2, 2]))
  warnings.warn(msg)


In [13]:
print(W_low, '\n',W_up, '\n',b_low, '\n',b_up)

[[[0. 1.]
  [1. 0.]]] 
 [[[0. 1.]
  [1. 0.]]] 
 [[0. 0.]] 
 [[0. 0.]]


In [14]:
modelgs(input)

tensor([[-2.,  2.]], device='cuda:0')

In [15]:
W_low[0,:] @ K.convert_to_numpy(input)[0,:] + b_low

array([[-2.,  2.]], dtype=float32)

In [16]:
W_up[0,:] @ K.convert_to_numpy(input)[0,:] + b_up

array([[-2.,  2.]], dtype=float32)

## Cas 2 z0 <= 0 <= z1

In [17]:
input = K.expand_dims(K.array([-3, 5], dtype='float32'), axis=0)
low = K.expand_dims(K.array([-4, 1], dtype='float32'), axis=0)
up = K.expand_dims(K.array([3, 5], dtype='float32'), axis=0)
bounds =  K.expand_dims(K.concatenate((low,up), axis=0), axis=0)

In [18]:
print(low.shape, bounds.shape)

torch.Size([1, 2]) torch.Size([1, 2, 2])


In [19]:
W_low, b_low, W_up, b_up = decomon_model.predict(bounds)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


In [20]:
print(W_low, '\n',W_up, '\n',b_low, '\n',b_up)

[[[0.8181818  0.18181819]
  [0.         1.        ]]] 
 [[[1.         0.        ]
  [0.18181819 0.8181818 ]]] 
 [[-1.6363636  0.       ]] 
 [[0.        1.6363636]]


In [21]:
print(W_low.shape)

(1, 2, 2)


On voit dans le code suivant qu'il y a un souci :

In [22]:
modelgs(input)

tensor([[-3.,  5.]], device='cuda:0')

In [23]:
W_low[0,:] @ K.convert_to_numpy(input)[0,:] + b_low

array([[-3.181818,  5.      ]], dtype=float32)

In [24]:
W_up[0,:] @ K.convert_to_numpy(input)[0,:] + b_up

array([[-3.      ,  5.181818]], dtype=float32)

In [25]:
from lipschitz_custom_tools import *

# Hybrid

In [22]:
input = K.expand_dims(K.array([-3, 5, 2, -2], dtype='float32'), axis=0)
low = K.expand_dims(K.array([-4, 1, 1, -3], dtype='float32'), axis=0)
up = K.expand_dims(K.array([3, 5, 3, -1], dtype='float32'), axis=0)
bounds =  K.expand_dims(K.concatenate((low,up), axis=0), axis=0)

In [23]:
bounds.shape

torch.Size([1, 2, 4])

In [24]:
modelgs = Sequential([GroupSort2()])

In [25]:
modelgs(input)

tensor([[-3.,  5., -2.,  2.]], device='cuda:0')

In [26]:
decomon_model = clone(modelgs, mapping_keras2decomon_classes={GroupSort2:DecomonGroupSort2}, final_ibp=False, final_affine=True, method='crown')  

In [27]:
W_low, b_low, W_up, b_up = decomon_model.predict(bounds)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


/home/aws_install/miniconda3/envs/k3torchenv/lib/python3.10/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['perturbation_domain_input_sequential_1']
Received: inputs=Tensor(shape=torch.Size([1, 2, 4]))
  warnings.warn(msg)


In [28]:
print(W_low, '\n',W_up, '\n',b_low, '\n',b_up)

[[[0.8181818  0.18181819 0.         0.        ]
  [0.         1.         0.         0.        ]
  [0.         0.         0.         1.        ]
  [0.         0.         1.         0.        ]]] 
 [[[1.         0.         0.         0.        ]
  [0.18181819 0.8181818  0.         0.        ]
  [0.         0.         0.         1.        ]
  [0.         0.         1.         0.        ]]] 
 [[-1.6363636  0.         0.         0.       ]] 
 [[ 0.         1.6363636  0.        -0.       ]]
